# Embedding Translations


see `plan.md` for more info


# 01 Setup


In [ ]:
### LOCAL INSTALLS ###
%pip install torch
%pip install torchvision
%pip install torchaudio
%pip install datasets
%pip install click
%pip install scikit-learn
%pip install plotly
%pip install seaborn

In [ ]:
### GENERAL IMPORTS ###

### ML IMPORTS ###
import torch

from src.viz.owl_metrics import calculate_metric


print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


device = torch.device("cuda:0")
# assert torch.cuda.is_available()

In [3]:
"""owler metrics document."""

import random
from typing import Any

import click
import numpy as np
import plotly.express as px
import torch
from sklearn.neighbors import NearestNeighbors
from torch.nn.functional import cosine_similarity
from torch.nn.functional import pairwise_distance


COSINE = "cosine"
EUCLIDEAN = "euclidean"
JACCARD = "jaccard"
RANK = "rank"
CKA = "cka"
AVAILABLE_METRICS = [COSINE, EUCLIDEAN, JACCARD, RANK, CKA]
MATCH_DIM_METRICS = [COSINE, EUCLIDEAN]
NEAREST_NEIGHBORS = [JACCARD, RANK]


def _pairwise_similarity(
    embeds1: torch.Tensor, embeds2: torch.Tensor, metric: str, device: torch.device
) -> torch.Tensor:
    """Calculates the pairwise cosine similarity or Euclidean distance between the given tensors.

    :param embeds1: The first tensors of dimension (N, D).
    :param embeds2: The second tensors of dimension (N, D).
    :param metric: The metric to be used.
    :param device: The device on which calculations are performed.
    :return: The calculated pairwise scores.
    """
    if metric == COSINE:
        return cosine_similarity(embeds1.to(device), embeds2.to(device))
    if metric == EUCLIDEAN:
        return pairwise_distance(embeds1.to(device), embeds2.to(device))
    raise NotImplementedError(
        f"Provided unsupported metric {metric} for pairwise similarity!"
    )


def _mean_pairwise_similarity(
    embeds1: torch.Tensor,
    embeds2: torch.Tensor,
    metric: str,
    batch_size: int,
    device: torch.device,
) -> torch.Tensor:
    """Calculates the pairwise cosine similarity or Euclidean distance between batches of the given tensors and sums them
    up. In the end, the mean score is returned.

    :param embeds1: A tensor of dimension (N, D).
    :param embeds2: A tensor of dimension (N, D).
    :param metric: The metric to use for calculating the pairwise scores.
    :param batch_size: The batch size used to split the tensors.
    :param device: The device on which calculations should be performed.
    :return: The mean score over all batches.
    """
    assert embeds1.shape == embeds2.shape
    embeds1_batches = embeds1.split(batch_size)
    embeds2_batches = embeds2.split(batch_size)
    total = 0
    sims = []
    for embed1, embed2 in zip(embeds1_batches, embeds2_batches, strict=False):
        sim = _pairwise_similarity(embed1, embed2, metric, device)
        sims = sims + sim.detach().cpu().tolist()
        total += torch.sum(sim).detach().cpu()
    text = "Cosine Similarities" if metric is COSINE else "Euclidean Distance"
    fig = px.histogram(x=sims, labels={"x": text}, title=f"{text} Distribution")
    return total / len(embeds1), fig


def _jaccard_sim(indices1: np.ndarray, indices2: np.ndarray) -> np.float64:
    """Calculates the Jaccard similarity between two 2D arrays by dividing the number of overlapping entries by the union
    of entries per row. The mean score over all rows is returned in the end.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The mean Jaccard similarity.
    """
    inds = np.concatenate((indices1, indices2), axis=1)
    len_union = np.array([len(np.unique(i)) for i in inds])
    len_intersection = np.array(
        [
            len(set(i).intersection(set(j)))
            for i, j in zip(indices1, indices2, strict=False)
        ]
    )
    return np.mean(len_intersection / len_union)


def _get_rank_sum(indices1: np.ndarray, indices2: np.ndarray) -> Any:
    """Computes the sum term for rank similarity given the two 1D-arrays containing the indices of the k-nearest neighbors
    of two sets of activations.

    :param indices1: One row of indices calculated for the first set of activations.
    :param indices2: One row of indices calculated for the second set of activations.
    :return: The calculated rank sum.
    """
    aux = np.concatenate((indices1, indices2))
    aux_sort_indices = np.argsort(aux, kind="mergesort")
    aux = aux[aux_sort_indices]
    mask = aux[1:] == aux[:-1]
    ar1_indices = aux_sort_indices[:-1][mask] + 1
    ar2_indices = aux_sort_indices[1:][mask] - indices1.size + 1
    rank_sum = np.sum(
        [
            2 / ((1 + abs(i - j)) * (i + j))
            for i, j in zip(ar1_indices, ar2_indices, strict=False)
        ]
    )
    return rank_sum


def _rank_sim(indices1: np.ndarray, indices2: np.ndarray):
    """Computes the rank similarity between two sets of indices. Rank similarities are calculated for each pair of rows and
    averaged.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The mean rank similarity.
    """
    rank_sums = [_get_rank_sum(i, j) for i, j in zip(indices1, indices2, strict=False)]
    len_intersection = np.array(
        [
            len(set(i).intersection(set(j)))
            for i, j in zip(indices1, indices2, strict=False)
        ]
    )
    factors = []
    for idx, elem1 in enumerate(len_intersection):
        if elem1 > 0:
            factors.append(1 / sum([1 / (i + 1) for i in range(int(elem1))]))
        else:
            factors.append(0)
    res = np.array(factors) * np.array(rank_sums)
    return np.mean(res)


def nn_sim(indices1: np.ndarray, indices2: np.ndarray, metric: str) -> Any:
    """Calculates the similarity of two sets of indices representing the index of nearest neighbors using Jaccard or rank
    similarity.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The average similarity.
    """
    if metric == JACCARD:
        return _jaccard_sim(indices1, indices2)
    return _rank_sim(indices1, indices2)


def _nearest_neighbors(
    embeds1: np.ndarray,
    embeds2: np.ndarray,
    queries1: np.ndarray,
    queries2: np.ndarray,
    metric: str,
    k: int,
    nn_function: str,
    baseline: bool = False,
) -> tuple[Any, None]:
    """Calculates the nearest neighbors for two sets of queries and returns the k indices of the closest embeddings for
    each query. After obtaining the indices, their similarity is calculated.

    :param embeds1: The first set of embeddings of shape (N, D1).
    :param embeds2: The second set of embeddings of shape (N, D2).
    :param queries1: The first set of queries of shape (M, D1).
    :param queries2: The second set of queries of shape (M, D2).
    :param metric: The metric to use for comparing nearest neighbors.
    :param k: The number of nearest neighbors to retrieve.
    :param nn_function: The metric to use for finding nearest neighbors.
    :param baseline: Whether to compute a baseline score, default: False.
    :return: A list of similarity scores of length k.
    """
    neigh1 = NearestNeighbors(n_neighbors=k, metric=nn_function, algorithm="brute")
    neigh1.fit(embeds1)

    neigh2 = NearestNeighbors(n_neighbors=k, metric=nn_function, algorithm="brute")
    neigh2.fit(embeds2)

    indices1 = neigh1.kneighbors(queries1, n_neighbors=k, return_distance=False)
    indices2 = neigh2.kneighbors(queries2, n_neighbors=k, return_distance=False)

    if baseline:
        np.random.shuffle(indices1)

    sims = []

    for i in range(k):
        j = i + 1
        sims.append(nn_sim(indices1[:, :j], indices2[:, :j], metric))
    return sims, None


# Code taken from https://haydn.fgl.dev/posts/a-better-index-of-similarity/
def _cka(A: torch.Tensor, B: torch.Tensor) -> tuple[Any, None]:
    # Mean center each neuron
    A = A - torch.mean(A, dim=0, keepdim=True)
    B = B - torch.mean(B, dim=0, keepdim=True)

    dot_product_similarity = torch.linalg.norm(torch.matmul(A.t(), B)) ** 2

    normalization_x = torch.linalg.norm(torch.matmul(A.t(), A))
    normalization_y = torch.linalg.norm(torch.matmul(B.t(), B))

    cka = dot_product_similarity / (normalization_x * normalization_y)

    dot_product_similarity.detach()
    normalization_x.detach()
    normalization_y.detach()
    A.detach()
    B.detach()
    del dot_product_similarity, normalization_x, normalization_y, A, B
    return cka, None


def _calculate_embed_metric(
    embeds1: torch.Tensor,
    embeds2: torch.Tensor,
    queries1: np.ndarray,
    queries2: np.ndarray,
    metric: str,
    batch_size: int,
    device: torch.device,
    k: int,
    nn_function: str,
    baseline: bool = False,
):
    """Calculates the similarity between two embedding matrices using the given metric.

    :param embeds1: Document embedding tensor of shape (N, D1).
    :param embeds2: Document embedding tensor of shape (N, D1) if the metric requires matching dimensions or (N, D2).
    :param queries1: Embedding of queries of shape (N, D1).
    :param queries2: Embedding of queries of shape (N, D2).
    :param metric: The metric to be used, required to be one of AVAILABLE_METRICS.
    :param batch_size: The batch size used to calculate pairwise similarity.
    :param device: The device on which calculations should be performed.
    :param k: The number of nearest neighbors to retrieve for Jaccard or rank similarity.
    :param nn_function: The function for determining nearest neighbors.
    :param baseline: Whether to compute a baseline score, default: False.
    :return: The calculated similarity.
    """
    assert len(embeds1) == len(embeds2)

    if metric in MATCH_DIM_METRICS:
        return _mean_pairwise_similarity(embeds1, embeds2, metric, batch_size, device)
    if metric in NEAREST_NEIGHBORS:
        return _nearest_neighbors(
            np.array(embeds1),
            np.array(embeds2),
            queries1,
            queries2,
            metric,
            k,
            nn_function,
            baseline,
        )
    if metric == CKA:
        return _cka(embeds1, embeds2)
    raise NotImplementedError(
        f"Provided unsupported metric {metric} for embedding similarity!"
    )


def _sample_embeddings(
    embeds1: torch.Tensor,
    embeds2: torch.Tensor,
    min_size: int,
    num_embeds: int,
    baseline: bool = False,
):
    """Returns a subset of embeddings if the desired number of embeddings to be compared is lower than the available ones.
    If a baseline score should be computed, the first set of embeddings is shuffled randomly if all embeddings are used
    or two different random subsets are returned.

    :param embeds1: Document embedding tensor of shape (N, D1).
    :param embeds2: Document embedding tensor of shape (N, D1) if the metric requires matching dimensions or (N, D2).
    :param min_size: The maximum number of embeddings that can be compared.
    :param num_embeds: The number of embeddings that should be compared.
    :param baseline: Whether a baseline score should be computed.
    """
    if baseline:
        indices = torch.randperm(embeds1.shape[0])
        embeds1 = embeds1[indices]
    if num_embeds > 0:
        if min_size < num_embeds:
            click.echo(
                f"Chosen number of embeddings is larger than number of available ones {min_size}. "
                f"Using all available."
            )
        else:
            indices = random.sample(range(min_size), num_embeds)
            embeds1 = embeds1[indices]
            embeds2 = embeds2[indices]
    return embeds1, embeds2


def calculate_metric(
    embeds1: torch.Tensor,
    embeds2: torch.Tensor,
    queries1: np.ndarray,
    queries2: np.ndarray,
    metric: str,
    batch_size: int,
    device: torch.device,
    num_embeds: int,
    k: int,
    nn_function: str,
    center: bool = False,
    baseline: bool = False,
):
    """Calculates the similarity between document embeddings after pre-processing the embedding vectors.

    :param embeds1: Document embedding tensor of shape (N, D1).
    :param embeds2: Document embedding tensor of shape (N, D1) if the metric requires matching dimensions or (N, D2).
    :param queries1: Embedding of queries of shape (N, D1).
    :param queries2: Embedding of queries of shape (N, D2).
    :param metric: The metric to be used, required to be one of AVAILABLE_METRICS.
    :param batch_size: The batch size used to calculate pairwise similarity.
    :param device: The device on which calculations should be performed.
    :param num_embeds: The number of embeddings to be compared.
    :param k: The number of nearest neighbors to retrieve for Jaccard or rank similarity.
    :param nn_function: The function for determining nearest neighbors.
    :param center: Whether embeddings should be mean-centered, default: False.
    :param baseline: Whether to compute a baseline score, default: False.
    :return: The calculated similarity.
    """
    min_size = min(len(embeds1), len(embeds2))
    assert k <= min_size
    embeddings1 = embeds1[:min_size]
    embeddings2 = embeds2[:min_size]
    embeddings1, embeddings2 = _sample_embeddings(
        embeddings1, embeddings2, min_size, num_embeds, baseline
    )

    if center:
        embeddings1 = embeddings1 - torch.mean(embeddings1, axis=0, keepdim=True)
        embeddings2 = embeddings2 - torch.mean(embeddings2, axis=0, keepdim=True)

    return _calculate_embed_metric(
        embeddings1,
        embeddings2,
        queries1,
        queries2,
        metric,
        batch_size,
        device,
        k,
        nn_function,
        baseline,
    )


def self_sim_score(metric: str):
    """Returns the similarity score between two identical vectors for the given metric.

    :param metric: The name of the metric.
    :return: The self-similarity score.
    """
    if metric == COSINE or metric == JACCARD or metric == RANK or metric == CKA:
        return 1
    if metric == EUCLIDEAN:
        return 0
    raise NotImplementedError(
        f"Cannot return self-similarity score of unsupported metric {metric}!"
    )

In [ ]:
from safetensors.torch import load_file


# Create sample embeddings (100 documents, 768 dimensions each)
embeds1 = load_file(
    "/Users/gat/work/FA2024/embedding_translation/data/openai_embeddings/text-embedding-3-large/fiqa/corpus_validation_embeddings.safetensors"
)["embeddings"]
embeds2 = load_file(
    "/Users/gat/work/FA2024/embedding_translation/data/openai_embeddings/text-embedding-3-small/fiqa/corpus_validation_embeddings.safetensors"
)["embeddings"]


similarity_score = _cka(embeds1, embeds2)[0]

print(f"Similarity score: {similarity_score}")
plot = False
if plot:  # Plot will be None for some metrics
    plot.show()

In [35]:
from pathlib import Path

from loguru import logger

from src.create_summaries_from_folder import EmbeddingDatasetInformation
from src.create_summaries_from_folder import ModelGenerator
from src.create_summaries_from_folder import anal_dump


__file__ = (
    "/Users/gat/work/FA2024/embedding_translation/src/embedding_translation.ipynb"
)

PROJ_ROOT = Path(__file__).parent.parent
logger.info(PROJ_ROOT.resolve())

embeddings_dir = PROJ_ROOT / "data" / "huggingface_embeddings"
dataset_infos = ModelGenerator.native_embedding_dataset_info_from_dir(embeddings_dir)
logger.info(len(dataset_infos))

[12/12/24 00:45:27] INFO     [2024095736.py:13] /Users/gat/work/FA2024/embedding_translation       ]8;id=443197;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2024095736.py\2024095736.py]8;;\:]8;id=705639;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2024095736.py#13\13]8;;\

                    INFO     [2024095736.py:17] 66                                                 ]8;id=679775;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2024095736.py\2024095736.py]8;;\:]8;id=787962;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2024095736.py#17\17]8;;\

In [37]:
# Group em
dataset_infos_grouped: dict[
    str, list[tuple[EmbeddingDatasetInformation, torch.Tensor]]
] = {}
for dataset_info in dataset_infos:
    if dataset_info.text_dataset_name in dataset_infos_grouped:
        pack = (dataset_info, load_file(dataset_info.dataset_filepath)["embeddings"])
        dataset_infos_grouped[dataset_info.text_dataset_name].append(pack)
    else:
        pack = (dataset_info, load_file(dataset_info.dataset_filepath)["embeddings"])
        dataset_infos_grouped[dataset_info.text_dataset_name] = [pack]

for text_dataset_name, datasets in dataset_infos_grouped.items():
    print(text_dataset_name)
    print(len(datasets))
    print(type(datasets[0][0]), type(datasets[0][1]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:9                                                                              │
│                                                                                                  │
│    6 │   │   pack = (dataset_info, load_file(dataset_info.dataset_filepath)["embeddings"])       │
│    7 │   │   dataset_infos_grouped[dataset_info.text_dataset_name].append(pack)                  │
│    8 │   else:                                                                                   │
│ ❱  9 │   │   pack = (dataset_info, load_file(dataset_info.dataset_filepath)["embeddings"])       │
│   10 │   │   dataset_infos_grouped[dataset_info.text_dataset_name] = [pack]                      │
│   11                                                                                             │
│   12 for text_dataset_name, datasets in dataset_infos_grouped.items():                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'load_file' is not defined

In [34]:
print(datasets)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 print(datasets)                                                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'datasets' is not defined

In [ ]:
# Add random cka to all dataset_infos
def add_random(dataset_infos_grouped):
    for text_dataset_name, datasets in dataset_infos_grouped.items():
        print(text_dataset_name)
        print(len(datasets))
        random_name = "Normal(0, 1)"
        if random_name in [dataset[0].embedding_model_name for dataset in datasets]:
            continue
        info = datasets[0][0].model_copy()
        info.dataset_filepath = None
        info.embedding_model_name = "Normal(0, 1)"
        embed = torch.randn_like(datasets[0][1])
        datasets.append((info, embed))
        print(len(datasets))


add_random(dataset_infos_grouped=dataset_infos_grouped)

In [ ]:
num_embeds = len(dataset_infos)
cka_matrices = {}

for text_dataset_name, datasets in dataset_infos_grouped.items():
    if text_dataset_name != "arguana":
        continue
    cka_matrix = [[None for _ in datasets] for _ in datasets]
    print(text_dataset_name)
    for i, (info1, embeds1) in enumerate(datasets):
        print(f"{i} of {len(datasets)}")
        for j, (info2, embeds2) in enumerate(datasets):
            sim = _cka(embeds1, embeds2)[0]
            cka_matrix[i][j] = sim
    cka_matrices[text_dataset_name] = torch.Tensor(cka_matrix)

print(cka_matrices)

In [ ]:
from src.viz.plot_heatmap import save_figure
from src.viz.plot_heatmap import visualize_heatmap


for text_dataset_name, datasets in dataset_infos_grouped.items():
    if text_dataset_name != "arguana":
        continue
    labels = [info.embedding_model_name for info, embeds in datasets]
    fig = visualize_heatmap(
        matrix=cka_matrices[text_dataset_name],
        config={
            "row_labels": labels,
            "col_labels": labels,
            "title": f"CKA Matrix on {text_dataset_name}",
            "xaxis_title": "Embedding Space 01",
            "yaxis_title": "Embedding Space 02",
        },
    )

    # fig.update_yaxes(autorange=True)
    fig["layout"]["yaxis"]["autorange"] = "reversed"
    fig.update_xaxes(side="top")

    save_figure(
        fig,
        f"cka_matrix_on_{text_dataset_name}",
        output_dir=PROJ_ROOT / "data" / "figs",
    )

In [33]:
from collection_utils import model2model_dimension
from src.create_summaries_from_folder import ModelGenerator
from src.create_summaries_from_folder import TrainingLogs
from src.create_summaries_from_folder import save_figure
from src.create_summaries_from_folder import visualize_heatmap


### GET TRAINING LOGS ###
train_loss_dir = PROJ_ROOT / "data" / "arguana_loss"
train_logs_dir = PROJ_ROOT / "data" / "arguana_loss_logs"

# Cached
if train_logs_dir.exists():
    train_values = ModelGenerator.load_data_as_train_logs(
        train_logs_dir / "arguana_loss_logs.json"
    )
    train_values = train_values.model_dump()["logs"]
# Load
else:
    train_logs_dir.mkdir(parents=True, exist_ok=False)
    train_values = ModelGenerator.get_train_logs(PROJ_ROOT / "data" / "arguana_loss")
    anal_dump(
        TrainingLogs(logs=train_values), "arguana_loss_logs", "data/arguana_loss_logs"
    )

logger.info(f"Received {len(train_values)} training values")
matrix = ModelGenerator.get_mapping_from_train_values(train_values)

# Create label dictionaries with relevant fields
labels = [
    {
        "org": label.split("/")[0],
        "dim": model2model_dimension(label),
        "model": label.split("/")[1],
        "full": label,
    }
    for label in matrix
]

# Sort by specified order (can be changed easily)
sort_order = ["dim", "org", "model"]
labels = sorted(labels, key=lambda x: tuple(x[k] for k in sort_order))

# Format display labels using the sorted dictionaries
display_labels = [
    f"({label['dim']}) {label['org']}/{label['model']}" for label in labels
]
models = [f"{label["full"]}" for label in labels]
sorted_by = f"Sorted by {', then '.join(sort_order)}"

for text_dataset_name in ["arguana"]:
    architecture = train_values[0]["info"]["mode"]
    ### MSE MATRIX ###
    mse_matrix = ModelGenerator.get_mse_matrix_from_matrix(matrix, models)
    fig = visualize_heatmap(
        matrix=mse_matrix,
        config={
            "row_labels": display_labels,
            "col_labels": display_labels,
            "title": f"MSE Matrix on {text_dataset_name}, (Architecture: {architecture})\n{sorted_by}",
            "xaxis_title": "Native Embedding Space",
            "yaxis_title": "Target Embedding Space",
            "value_format": ".2e",
        },
    )

    fig["layout"]["yaxis"]["autorange"] = "reversed"
    fig.update_xaxes(side="top")

    save_figure(
        fig, f"mse_matrix_on{text_dataset_name}", output_dir=PROJ_ROOT / "data" / "figs"
    )

    ### MAE MATRIX ###
    mae_matrix = ModelGenerator.get_mae_matrix_from_matrix(matrix, models)
    fig = visualize_heatmap(
        matrix=mae_matrix,
        config={
            "row_labels": display_labels,
            "col_labels": display_labels,
            "title": f"MAE Matrix on {text_dataset_name}  (Architecture: {architecture})\n{sorted_by}",
            "xaxis_title": "Native Embedding Space",
            "yaxis_title": "Target Embedding Space",
            "value_format": ".2e",
        },
    )

    fig["layout"]["yaxis"]["autorange"] = "reversed"
    fig.update_xaxes(side="top")

    save_figure(
        fig, f"mae_matrix_on{text_dataset_name}", output_dir=PROJ_ROOT / "data" / "figs"
    )
    # logger.info(mae_matrix)
    logger.info("SUCCESS")

[12/11/24 16:17:11] INFO     [2819754872.py:22] Received 210 training values                       ]8;id=965542;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2819754872.py\2819754872.py]8;;\:]8;id=806670;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2819754872.py#22\22]8;;\

                    INFO      Starting heatmap visualization process                            ]8;id=896866;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py\plot_heatmap.py]8;;\:]8;id=625029;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py#115\115]8;;\

                    INFO      Creating heatmap visualization...                                  ]8;id=355872;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py\plot_heatmap.py]8;;\:]8;id=713512;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py#40\40]8;;\

                    SUCCESS   Heatmap created successfully                                       ]8;id=888199;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py\plot_heatmap.py]8;;\:]8;id=466257;file:///Users/gat/work/FA2024/embedding_translation/src/viz/plot_heatmap.py#89\89]8;;\

                    INFO      Saving HTML figure to                                               ]8;id=700513;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py\save_figure.py]8;;\:]8;id=198006;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py#32\32]8;;\
                             /Users/gat/work/FA2024/embedding_translation/data/figs/html/mse_matr                  
                             ix_onarguana.html                                                                     

...

                    INFO      Saving HTML figure to                                               ]8;id=589489;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py\save_figure.py]8;;\:]8;id=254329;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py#32\32]8;;\
                             /Users/gat/work/FA2024/embedding_translation/data/figs/html/mae_matr                  
                             ix_onarguana.html                                                                     

                    SUCCESS   Successfully saved HTML to                                          ]8;id=889348;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py\save_figure.py]8;;\:]8;id=632331;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py#40\40]8;;\
                             /Users/gat/work/FA2024/embedding_translation/data/figs/html/mae_matr                  
                             ix_onarguana.html                                                                     

                    INFO      Saving PNG figure to                                                ]8;id=362105;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py\save_figure.py]8;;\:]8;id=299170;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py#67\67]8;;\
                             /Users/gat/work/FA2024/embedding_translation/data/figs/imgs/mae_matr                  
                             ix_onarguana.png                                                                      

                    SUCCESS   Successfully saved PNG to                                           ]8;id=843717;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py\save_figure.py]8;;\:]8;id=780607;file:///Users/gat/work/FA2024/embedding_translation/src/viz/save_figure.py#76\76]8;;\
                             /Users/gat/work/FA2024/embedding_translation/data/figs/imgs/mae_matr                  
                             ix_onarguana.png                                                                      

                    INFO     [2819754872.py:98] SUCCESS                                            ]8;id=475340;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2819754872.py\2819754872.py]8;;\:]8;id=752952;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_80562/2819754872.py#98\98]8;;\